In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr2_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('Dataset/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select('features', 'Yearly Amount Spent')

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                341|
|   mean|  500.7559515512817|
| stddev|  76.94661773169919|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.605491060600116|
|-4.6700702113944885|
| 1.0896392671383524|
| -17.36351556598595|
|  8.210847287269871|
| -1.215748255538415|
|-2.8397405069242723|
|-3.8673565255774065|
| 2.4585825270339114|
| -6.619299955519409|
| -4.972804006608271|
|-1.4541530930208637|
| 18.387892528518023|
| -5.441947794634245|
|  -9.67542878891112|
|  8.833812224414999|
| -5.165020386420736|
|  -16.9429337483719|
| 11.771682378621392|
| -13.78240151801674|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

9.650355922721053

In [24]:
test_results.r2

0.986824025517287

In [25]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.9716756438877...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9262720263601...|
|[31.9365486184489...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = lr_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.03486001202737|
|[30.4925366965402...|  287.141315931309|
|[30.5743636841713...| 440.9747744909273|
|[30.8162006488763...|283.44985651445495|
|[30.9716756438877...|486.42776246962285|
|[31.2606468698795...| 422.5423795124898|
|[31.2681042107507...| 426.3102736807482|
|[31.4252268808548...| 534.6340751803393|
|[31.4459724827577...|482.41838240809466|
|[31.5147378578019...| 496.4317879519808|
|[31.5257524169682...|448.93843081649015|
|[31.5761319713222...| 542.6807370823492|
|[31.6005122003032...| 460.7849589625789|
|[31.6253601348306...|381.77884855155844|
|[31.8279790554652...|449.67817633585264|
|[31.8512531286083...| 464.1584344423834|
|[31.8745516945853...|397.45026463268823|
|[31.9048571310136...|490.89279117118804|
|[31.9262720263601...|  380.433251065705|
|[31.9365486184489...|440.98178641334493|
+--------------------+------------

In [30]:
predictions.describe().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|               159|
|   mean| 496.7835683563227|
| stddev| 83.52752933528046|
|    min|283.44985651445495|
|    max| 684.4321982197789|
+-------+------------------+

